In [2]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os, re, warnings
import gensim
from gensim.models import Word2Vec
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier

In [3]:
df = pd.read_csv('spotify_songs.csv')
df.head()

,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,6,-2.634,1,0.0583,0.1020,0.000000,0.0653,0.518,122.036,194754
1,0r7CVbZTWZgbTCYdfa2P31,Memories - Dillon Francis Remix,Maroon 5,67,63rPSO264uRjW1X5E6cWv6,Memories (Dillon Francis Remix),2019-12-13,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,11,-4.969,1,0.0373,0.0724,0.004210,0.3570,0.693,99.972,162600
2,1z1Hg7Vb0AhHDiEmnDE79l,All the Time - Don Diablo Remix,Zara Larsson,70,1HoSmj2eLcsrR0vE9gThr4,All the Time (Don Diablo Remix),2019-07-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,-3.432,0,0.0742,0.0794,0.000023,0.1100,0.613,124.008,176616
3,75FpbthrwQmzHlBJLuGdC7,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,1nqYsOef1yKKuGOVchbsk6,Call You Mine - The Remixes,2019-07-19,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,7,-3.778,1,0.1020,0.0287,0.000009,0.2040,0.277,121.956,169093
4,1e8PAfcKUYoKkxPhrHqw4x,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,7m7vv9wlQ4i0LFuJiE2zsQ,Someone You Loved (Future Humans Remix),2019-03-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,-4.672,1,0.0359,0.0803,0.000000,0.0833,0.725,123.976,189052


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32833 entries, 0 to 32832
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   track_id                  32833 non-null  object 
 1   track_name                32828 non-null  object 
 2   track_artist              32828 non-null  object 
 3   track_popularity          32833 non-null  int64  
 4   track_album_id            32833 non-null  object 
 5   track_album_name          32828 non-null  object 
 6   track_album_release_date  32833 non-null  object 
 7   playlist_name             32833 non-null  object 
 8   playlist_id               32833 non-null  object 
 9   playlist_genre            32833 non-null  object 
 10  playlist_subgenre         32833 non-null  object 
 11  danceability              32833 non-null  float64
 12  energy                    32833 non-null  float64
 13  key                       32833 non-null  int64  
 14  loudne

In [5]:
playlists = df.groupby('playlist_id')['track_name'].apply(list)

In [6]:
playlists

playlist_id
0275i1VNfBnsNbPl0QIBpG    [Uptown (feat. Bun B & Lil Wayne), All Girls A...
03qQtbNHoJuFezRu2CnLuF    [Rock You Like a Hurricane - 2015 - Remaster, ...
03sDEv7FN58Mb9CJOs1Tgn    [Malas Intenciones, Que No Pare el Reggaeton, ...
06zrBJ5cts5aemZmqe80J7    [Backfire, Like I Love You, For You, Hey, Feel...
07SNJ4MwYba9wwmzrbjmYi    [Freedom & Fireworks, Say Something, Heartbeat...
                                                ...                        
7sq6nuruoMyDhEWkX2oYOg    [Light - Deep Mix, Don't Start Now - Kungs Rem...
7tkgK1tm9hYkWp7EFyOcAr    [Life Is Good (feat. Drake), HIGHEST IN THE RO...
7vJOXFe40axY7qS39vGDyH    [Animals - Radio Edit, Bailar (feat. Elvis Cre...
7xWdFCrU5Gka6qp1ODrSdK    [Life Is Music, Make Believe, The King (with D...
7xWuNevFBmwnFEg6wzdCc7    [La Playa - Remix, Porfa, Doble Cara, En Cero ...
Name: track_name, Length: 471, dtype: object

## Essaierons de deux manières
#### Première Manière : On recuperer les tracks name des artists par playlists_id 
#### deuxieme maniere: On recupère le nom des artists et leurs tracks_name

## GOAL : L'objectif est de mettre en place un système de recommandations des chansons de spotify. 
Quand l'utilisateur entre dans l'application "Sing" et mets le nom d'un artiste quelconque pour écouter avant la 
fin de la playlist en cours, il obtiendra des suggestions de chansons similaires de l'artist et memes ses playlists. 

### La Technique : 
Nous allons mettre à disposition une application web interactive streamlit basé sur un model pre-trained(model pré-entrainé) qui chargera de fournir des suggestions à l'utilisateur afin de le satisfaire. Cette application ressemblera un peu comme à celle de spotify.

In [7]:
data = df[['track_name', 'track_artist']]
data.head()

,track_name,track_artist
0,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran
1,Memories - Dillon Francis Remix,Maroon 5
2,All the Time - Don Diablo Remix,Zara Larsson
3,Call You Mine - Keanu Silva Remix,The Chainsmokers
4,Someone You Loved - Future Humans Remix,Lewis Capaldi


In [8]:
data = data.rename(columns = {'track_name': 'song',
                      'track_artist': 'Name_artist'})

In [9]:
data.duplicated().sum()

6603

In [10]:
data = data.drop_duplicates()
data.head()

,song,Name_artist
0,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran
1,Memories - Dillon Francis Remix,Maroon 5
2,All the Time - Don Diablo Remix,Zara Larsson
3,Call You Mine - Keanu Silva Remix,The Chainsmokers
4,Someone You Loved - Future Humans Remix,Lewis Capaldi


### Pretraitement

In [11]:
data['song'] = data['song'].str.lower()
data['song'] = data['song'].str.replace('[^a-zA-Z0-9\s]', '', regex=True)

In [12]:
data.head()

,song,Name_artist
0,i dont care with justin bieber loud luxury remix,Ed Sheeran
1,memories dillon francis remix,Maroon 5
2,all the time don diablo remix,Zara Larsson
3,call you mine keanu silva remix,The Chainsmokers
4,someone you loved future humans remix,Lewis Capaldi


In [13]:
data.to_csv('data.csv', index = False)

In [14]:
sing = data.groupby('song')['Name_artist'].apply(list)

### Model of Word2Vec

In [15]:
modelvec = Word2Vec(sentences= sing, vector_size=200, workers=4, window=5,
                    min_count=1)

In [16]:
modelvec.save('recommended.model')

In [17]:
modelvec.wv.most_similar('Lewis Capaldi', topn= 5)

[('Jonasu', 0.27884289622306824),
 ('Omar', 0.2435751110315323),
 ('Ran the Man', 0.23476071655750275),
 ('Survivor', 0.2245892435312271),
 ('Sultan + Shepard', 0.21763363480567932)]